In [58]:
from google.colab import files
uploaded = files.upload()

filename=list(uploaded.keys())[0]

with open(filename, 'wb') as f:
    f.write(uploaded[filename])

In [0]:
!cat /proc/meminfo
!cat /proc/cpuinfo
!df -h


In [1]:
ls

accidents.dat  datalab/  retail.dat


In [0]:
import sys,csv,time


def open_file():
    with open('retail.dat','r') as fi:
        reader=csv.reader(fi)
        lines=list(reader)
    temp=[i[0].split(' ') for i in lines]
    rows=[i[:-1] for i in temp]
    return rows

  
def pass1(baskets,s):
    items={}
    hasht={}
    
    for b in baskets:
      for i in range(len(b)):
        if b[i] not in items:
          items[b[i]]=1
        else:
          items[b[i]]+=1
        for j in range(i+1,len(b)):
          pair=(b[i],b[j])
          bucket=hash(pair)
          #bucket=int(b[i]+b[j])%N
          if bucket not in hasht:
            hasht[bucket]=1
          elif hasht[bucket]<s:
            hasht[bucket]+=1
    '''
    for b in baskets:
      for i in b:
        if i not in items:
          items[i]=1
        else:
          items[i]+=1
    print('C1:',len(items))
    for b in baskets:
      for i in range(len(b)-1):
        for j in range(i+1,len(b)):
          pair=(b[i],b[j])
          bucket=hash(pair)%N
          #bucket=int(b[i]+b[j])%N
          if bucket in hasht:
            hasht[bucket]+=1
          else:
            hasht[bucket]=1
    '''
    print('Hash Table:',len(hasht))
    rest=[i for i in hasht if hasht[i]>=s]
    print('More Than S:',len(rest))
    
    frequent_items=[(i,) for i in items if items[i] >= s]
    print('L1:',len(frequent_items))
    return frequent_items,hasht

  
def pass2(ffc,baskets,s,vector):
    candidates={}
    for i in range(len(ffc)-1):
        for j in range(i+1,len(ffc)):
          pair=ffc[i]+ffc[j]
          h=hash(pair)
          if h in vector:
            candidates[pair]=0
    print('C2'+':',len(candidates))
    for b in baskets:
        for c in candidates:
          if c[1] in b and c[0] in b:
            candidates[c]+=1

    freq_candidates=[i for i in candidates if candidates[i]>=s]
    print('L2'+':',len(freq_candidates))
    return freq_candidates
  
  
#ffc=former frequent candidates
#s=support
#K=the size of frequent items that will return 
def passk(K,ffc,baskets,s):
    assert(K>1),'Can Not Use Passk() At First Pass'
    def all_in_former(merged,ffc):
        sub={}
        merged=list(merged)
        for i in merged:
          temp=[i for i in merged]
          temp.remove(i)
          sub[tuple(temp)]=False  
        for c in ffc:
          for s in sub:
            if set(c)==set(s):
              sub[s]=True
        all_in=True
        for s in sub:
          if sub[s]==False:
            all_in=False   
        return all_in
      
    def not_duplicate(merged,candidates):
        not_in=True
        for t in candidates:
            if set(t)==set(merged):
                not_in=False
        return not_in
    
    candidates={}
    
    for i in range(len(ffc)-1):
        for j in range(i+1,len(ffc)):
            merged=tuple(set(ffc[i]+ffc[j]))
            if len(merged)==K:
                if not_duplicate(merged,candidates):
                    if all_in_former(merged,ffc):
                        candidates[merged]=0
    print('C'+str(K)+':',len(candidates))
                       
    check=[]
    t=list(candidates)
    for i in range(len(t)-1):
        for j in range(i+1,len(t)):
            if set(t[i]) == set(t[j]):
                check.append(t[j])
    assert(len(check)==0),'Exist Duplicate Candidate Elements'
    
    for b in baskets:
        for c in candidates:
            all_in_basket=True
            for i in c:
                if i not in b:
                    all_in_basket=False
                    break
            if all_in_basket:
                candidates[c]+=1
        
    freq_candidates=[i for i in candidates if candidates[i]>=s]
    print('L'+str(K)+':',len(freq_candidates))
    return freq_candidates
  
  
def pcy(K,sup_rate=0.01,confidence=0.5):
  start = time.clock()
  
  baskets=open_file()
  del baskets[0]
  print('Baskets:',len(baskets))
  support=sup_rate*len(baskets)
  print('Support:',support)
  
  singleton,hasht=pass1(baskets,support)
  
  
  for i in hasht:
    if hasht[i]>=support:
      hasht[i]=1
    else:
      hasht[i]=0
  vector=[i for i in hasht if hasht[i]==1]
  
  freq_items=singleton
  for i in range(2,K+1):
    if i==2:
        freq_items=pass2(freq_items,baskets,support,vector)
    else:
        freq_items=passk(i,freq_items,baskets,support)
  
  print ('Run Time:',time.clock()-start)
  return freq_items




In [8]:
test=pcy(5)

Baskets: 88161
Support: 881.61
Hash Table: 3586542
More Than S: 58
L1: 70
C2: 58
L2: 58
C3: 37
L3: 25
C4: 6
L4: 6
C5: 0
L5: 0
Run Time: 12.13500700000003


**test**

In [11]:
from math import *
start = time.clock()

baskets=open_file()
del baskets[0]
support=0.01*len(baskets)

candidates={}
'''
sort_baskets=[]
for i in baskets:
  sort_baskets.append(sorted(i))
'''
for b in baskets:
  for i in b:
    if i not in candidates:
        candidates[i]=1
    else:
        candidates[i]+=1

C1=len(candidates)
N=2**20
ap=[]
count=0
hasht={}

for b in baskets:
  for i in range(len(b)-1):
    for j in range(i+1,len(b)):
      pair=(b[i],b[j])
      bc=hash(pair)
      bucket=hash(pair)
      
      count+=1
      ap.append(bc)
      
      if bucket in hasht:
        hasht[bucket]+=1
      else:
        hasht[bucket]=1

singleton=[i for i in candidates if candidates[i]>=support]


print('all items:',C1)    
print('all possible pairs:',C1*(C1-1)//2)
print('actual pairs:',len(ap)+count)
print('unique pairs:',len(set(ap)))
print('hash   table:',len(hasht))
rest=[i for i in hasht if hasht[i]>=support]
print('more than  s:',len(rest))
print('singleton:',len(singleton))
print('C2=(L1 2)   :',len(singleton)*(len(singleton)-1)//2)

print ('Run Time:',time.clock()-start)

all items: 16470
all possible pairs: 135622215
actual pairs: 14327800
unique pairs: 3586542
hash   table: 3586542
more than  s: 58
singleton: 70
C2=(L1 2)   : 2415
Run Time: 14.706275000000005


In [0]:
all items: 468
all possible pairs: 109278
actual pairs: 190125806
unique pairs: 0
hash   table: 4096
more than  s: 3373
singleton: 166
C2=(L1 2)   : 13695
Run Time: 189.04417